# Octopy Features Generation predicting the current matches

Vamos a crear las features que me permite obtener Octopy estas son las siguientes:

* Coefs para equipo local y visitante
* Probs para equipo local y visitante

In [45]:
import warnings
warnings.filterwarnings('ignore')

import sqlite3
import pandas as pd
import time
from datetime import *
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from logistic_model import LogisticRegression, LogisticModel
from implied import ImpliedProbability
from sklearn.metrics import plot_confusion_matrix, accuracy_score

In [46]:
def connect_database(db):
    con = sqlite3.connect(db)
    cursor = con.cursor()
    return cursor, con


def leer_partidos(con):
    partidos = pd.read_sql_query('SELECT * FROM partidos', con) #, index_col='fecha')
    return partidos

cur, con = connect_database('../../Clasificacion.db')

partidos = leer_partidos(con)

In [47]:
partidos['fecha'] = partidos['timestamp'].map(lambda X: datetime.fromtimestamp(int(X)).date())
#partidos['fecha'] = partidos['timestamp']

#partidos['fecha'] = partidos['timestamp']
#partidos['fecha'] = partidos['fecha'].map(lambda X: int(datetime.fromtimestamp(int(X)).date().strftime('%Y%m%d')))
partidos = partidos.sort_values(by='fecha')

In [48]:
part_test = partidos[partidos['fecha'] >= date(2021, 10, 5)]
partidos = partidos[partidos.fecha < date(2021, 10, 6)]

In [49]:
part_test.head(15)

,id_partido,temporada,division,jornada,equipo_local,equipo_visitante,goles_local,goles_visitante,fecha,timestamp,...,visitante_puntos_antes,visitante_jugados_antes,visitante_ganados_antes,visitante_empatados_antes,visitante_perdidos_antes,visitante_goles_favor_antes,visitante_goles_contra_antes,visitante_racha_partidos,visitante_racha_gf,visitante_racha_gc
39848,50005,2021-22,1,9,Real Madrid,Atletico de Bilbao,0,0,2021-10-06,1633471200,...,13,8,3,4,1,7,4,None,None,None
39849,50006,2021-22,1,9,Levante,Getafe,0,0,2021-10-06,1633471200,...,1,8,0,1,7,3,13,None,None,None
39850,50007,2021-22,1,9,Real Sociedad,Mallorca,0,0,2021-10-06,1633471200,...,11,8,3,2,3,7,12,None,None,None
39851,50008,2021-22,1,9,Rayo Vallecano,Elche,0,0,2021-10-06,1633471200,...,9,8,2,3,3,5,8,None,None,None
39847,50004,2021-22,1,9,Granada,Atletico de Madrid,0,0,2021-10-06,1633471200,...,17,8,5,2,1,11,6,None,None,None
39846,50003,2021-22,1,9,Celta,Sevilla,0,0,2021-10-06,1633471200,...,14,7,4,2,1,10,3,None,None,None
39764,50013,2021-22,2,9,Las Palmas,Tenerife,0,0,2021-10-06,1633471200,...,14,8,4,2,2,10,7,None,None,None
39844,50001,2021-22,1,9,Alaves,Betis,0,0,2021-10-06,1633471200,...,12,8,3,3,2,11,9,None,None,None
39852,50009,2021-22,1,9,Villarreal,Osasuna,0,0,2021-10-06,1633471200,...,14,8,4,2,2,11,11,None,None,None
39765,50014,2021-22,2,9,Leganes,Valladolid,0,0,2021-10-06,1633471200,...,11,8,3,2,3,9,9,None,None,None


In [50]:
partidos.tail(15)

,id_partido,temporada,division,jornada,equipo_local,equipo_visitante,goles_local,goles_visitante,fecha,timestamp,...,visitante_puntos_antes,visitante_jugados_antes,visitante_ganados_antes,visitante_empatados_antes,visitante_perdidos_antes,visitante_goles_favor_antes,visitante_goles_contra_antes,visitante_racha_partidos,visitante_racha_gf,visitante_racha_gc
39837,39745,2021-22,1,8,Cadiz,Valencia,0,0,2021-10-02,1633125600,...,11,7,3,2,2,12,8,None,None,None
39836,39744,2021-22,1,8,Mallorca,Levante,1,0,2021-10-02,1633125600,...,4,7,0,4,3,6,12,None,None,None
39835,39743,2021-22,1,8,Osasuna,Rayo Vallecano,1,0,2021-10-02,1633125600,...,13,7,4,1,2,13,7,None,None,None
39753,39832,2021-22,2,8,Zaragoza,Oviedo,0,0,2021-10-02,1633125600,...,10,7,2,4,1,8,6,None,None,None
39842,39750,2021-22,1,8,Villarreal,Betis,2,0,2021-10-03,1633212000,...,12,7,3,3,1,11,7,None,None,None
39840,39748,2021-22,1,8,Espanol,Real Madrid,2,1,2021-10-03,1633212000,...,17,7,5,2,0,21,8,None,None,None
39839,39747,2021-22,1,8,Elche,Celta,1,0,2021-10-03,1633212000,...,7,7,2,1,4,7,10,None,None,None
39841,39749,2021-22,1,8,Getafe,Real Sociedad,1,1,2021-10-03,1633212000,...,16,7,5,1,1,10,6,None,None,None
39759,39838,2021-22,2,8,Leganes,Lugo,1,1,2021-10-03,1633212000,...,7,7,1,4,2,8,10,None,None,None
39843,39751,2021-22,1,8,Granada,Sevilla,1,0,2021-10-03,1633212000,...,14,6,4,2,0,10,2,None,None,None


In [51]:
temporadas = partidos['temporada'].unique()
dias = partidos['fecha'].unique()
divisiones = partidos['division'].unique()

In [7]:
for div in divisiones:
    partidos_div = partidos[partidos['division'] == div]

    for temp in temporadas:
        partidos_temp = partidos_div[partidos_div['temporada'] == temp]
        equipos = partidos_temp['equipo_local'].unique()
        dias = partidos_temp[partidos_temp['jornada'] > 2]['fecha'].unique()

        for d in dias:
            partidos_dia = partidos_temp[partidos_temp['fecha'] <= d]
            model = LogisticModel()
            model.fit(partidos_dia['equipo_local'],partidos_dia['equipo_visitante'],partidos_dia['goles_local'],partidos_dia['goles_visitante'])
            coefs = model.get_coef()
            partidos_hoy = partidos_dia[partidos_dia['fecha'] == d]
            for index, row in partidos_hoy.iterrows():
                partidos.at[partidos['id_partido'] == row.id_partido, 'local_home_wins'] = \
                    coefs.loc['home_'+row.equipo_local]['home wins'][0]
                partidos.at[partidos['id_partido'] == row.id_partido, 'local_draw'] = \
                    coefs.loc['home_'+row.equipo_local]['draw'][0]
                partidos.at[partidos['id_partido'] == row.id_partido, 'local_away_wins'] = \
                    coefs.loc['home_'+row.equipo_local]['away wins'][0]

                partidos.at[partidos['id_partido'] == row.id_partido, 'visitante_home_wins'] = \
                    coefs.loc['away_'+row.equipo_local]['home wins'][0]
                partidos.at[partidos['id_partido'] == row.id_partido, 'visitante_draw'] = \
                    coefs.loc['away_'+row.equipo_local]['draw'][0]
                partidos.at[partidos['id_partido'] == row.id_partido, 'visitante_away_wins'] = \
                    coefs.loc['away_'+row.equipo_local]['away wins'][0]

                partidos.at[partidos['id_partido'] == row.id_partido, 'probs_equipo_local'] = \
                    model.predict_proba(row.equipo_local, row.equipo_visitante)[row.equipo_local][0]
                partidos.at[partidos['id_partido'] == row.id_partido, 'probs_equipo_visitante'] = \
                    model.predict_proba(row.equipo_local, row.equipo_visitante)[row.equipo_visitante][0]
                partidos.at[partidos['id_partido'] == row.id_partido, 'probs_draw'] = \
                    model.predict_proba(row.equipo_local, row.equipo_visitante)['draw'][0]
            if div == 1:
                model1 = model
                coefs1 = coefs
            else:
                model2 = model
                coefs2 = coefs

        print(temp + ' ' + str(div))


1970-71 2
1971-72 2
1972-73 2
1973-74 2
1974-75 2
1975-76 2
1976-77 2
1977-78 2
1978-79 2
1979-80 2
1980-81 2
1981-82 2
1982-83 2
1983-84 2
1984-85 2
1985-86 2
1986-87 2
1987-88 2
1988-89 2
1989-90 2
1990-91 2
1991-92 2
1992-93 2
1993-94 2
1994-95 2
1995-96 2
1996-97 2
1997-98 2
1998-99 2
1999-00 2
2000-01 2
2001-02 2
2002-03 2
2003-04 2
2004-05 2
2005-06 2
2006-07 2
2007-08 2
2008-09 2
2009-10 2
2010-11 2
2011-12 2
2012-13 2
2013-14 2
2014-15 2
2015-16 2
2016-17 2
2017-18 2
2018-19 2
2019-20 2
2020-21 2
2021-22 2
1970-71 1
1971-72 1
1972-73 1
1973-74 1
1974-75 1
1975-76 1
1976-77 1
1977-78 1
1978-79 1
1979-80 1
1980-81 1
1981-82 1
1982-83 1
1983-84 1
1984-85 1
1985-86 1
1986-87 1
1987-88 1
1988-89 1
1989-90 1
1990-91 1
1991-92 1
1992-93 1
1993-94 1
1994-95 1
1995-96 1
1996-97 1
1997-98 1
1998-99 1
1999-00 1
2000-01 1
2001-02 1
2002-03 1
2003-04 1
2004-05 1
2005-06 1
2006-07 1
2007-08 1
2008-09 1
2009-10 1
2010-11 1
2011-12 1
2012-13 1
2013-14 1
2014-15 1
2015-16 1
2016-17 1
2017-18 1


In [8]:
partidos.to_csv('partidos_with_octopy_probabilites.csv')

In [9]:
model.predict_proba('Barcelona', 'Valencia')

,Valencia,draw,Barcelona
probability,0.194366,0.374059,0.431576


In [52]:
partidos_predict = part_test

for div in divisiones:
    if div == 1:
        model = model1
        coefs = coefs1
    else:
        model = model2
        coefs = coefs2
        
    part_test = partidos_predict[partidos_predict['division'] == div]

    for index, row in part_test.iterrows():
        print(temp + ' ' + str(div) + ' ' + row.equipo_local + '-' + row.equipo_visitante )
        part_test.at[part_test['id_partido'] == row.id_partido, 'local_home_wins'] = \
            coefs.loc['home_'+row.equipo_local]['home wins'][0]
        part_test.at[part_test['id_partido'] == row.id_partido, 'local_draw'] = \
            coefs.loc['home_'+row.equipo_local]['draw'][0]
        part_test.at[part_test['id_partido'] == row.id_partido, 'local_away_wins'] = \
            coefs.loc['home_'+row.equipo_local]['away wins'][0]

        part_test.at[part_test['id_partido'] == row.id_partido, 'visitante_home_wins'] = \
            coefs.loc['away_'+row.equipo_local]['home wins'][0]
        part_test.at[part_test['id_partido'] == row.id_partido, 'visitante_draw'] = \
            coefs.loc['away_'+row.equipo_local]['draw'][0]
        part_test.at[part_test['id_partido'] == row.id_partido, 'visitante_away_wins'] = \
            coefs.loc['away_'+row.equipo_local]['away wins'][0]

        part_test.at[part_test['id_partido'] == row.id_partido, 'probs_equipo_local'] = \
            model.predict_proba(row.equipo_local, row.equipo_visitante)[row.equipo_local][0]
        part_test.at[part_test['id_partido'] == row.id_partido, 'probs_equipo_visitante'] = \
            model.predict_proba(row.equipo_local, row.equipo_visitante)[row.equipo_visitante][0]
        part_test.at[part_test['id_partido'] == row.id_partido, 'probs_draw'] = \
            model.predict_proba(row.equipo_local, row.equipo_visitante)['draw'][0]
    
    if div == 1:
        part_test_1 = part_test
    else:
        part_test_2 = part_test


2021-22 2 Las Palmas-Tenerife
2021-22 2 Leganes-Valladolid
2021-22 2 Ponferradina-Fuenlabrada
2021-22 2 Girona-Huesca
2021-22 2 Sporting de Gijon-Alcorcon
2021-22 1 Real Madrid-Atletico de Bilbao
2021-22 1 Levante-Getafe
2021-22 1 Real Sociedad-Mallorca
2021-22 1 Rayo Vallecano-Elche
2021-22 1 Granada-Atletico de Madrid
2021-22 1 Celta-Sevilla
2021-22 1 Alaves-Betis
2021-22 1 Villarreal-Osasuna
2021-22 1 Barcelona-Valencia
2021-22 1 Espanol-Cadiz


In [53]:
part_test_1.to_csv('jornada_with_octopy_probabilites_1.csv')
part_test_2.to_csv('jornada_with_octopy_probabilites_2.csv')

## Convert String Date to Timestamp

In [34]:
import time
import datetime

In [35]:
time.mktime(datetime.datetime.strptime("16/10/2021", "%d/%m/%Y").timetuple())

1634335200.0